In [1]:
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
import numpy as np

In [2]:
# Dataset: 10 features, 2 classes
x, y = make_classification(n_samples=1000, n_features=10, n_classes=2)

In [3]:
rf = RandomForestClassifier()
print(np.mean(cross_val_score(rf, x, y, cv=20, scoring='roc_auc')))

0.9873191666666665


In [4]:
def rf_cv(n_estimators, min_samples_split, max_features, max_depth):
    val = cross_val_score(
        RandomForestClassifier(n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_features=min(max_features, 0.999), # float
            max_depth=int(max_depth),
            random_state=2
        ),
        x, y, scoring='roc_auc', cv=5
    ).mean()
    return val

In [11]:
rf_bo = BayesianOptimization(
    f=rf_cv,
    pbounds={
        'n_estimators': (10, 250),
        'min_samples_split': (2, 25),
        'max_features': (0.1, 0.999),
        'max_depth': (5, 15)
    },
    random_state=1
)

rf_bo.maximize(
    # init_points=3,
    # n_iter=3
)

|   iter    |  target   | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------
| 1         | 0.9835    | 9.17      | 0.7476    | 2.003     | 82.56     |
| 2         | 0.9672    | 6.468     | 0.183     | 6.284     | 92.93     |
| 3         | 0.9843    | 8.968     | 0.5844    | 11.64     | 174.5     |
| 4         | 0.9831    | 7.045     | 0.8894    | 2.63      | 170.9     |
| 5         | 0.9836    | 9.173     | 0.6023    | 5.229     | 57.54     |
| 6         | 0.9806    | 9.957     | 0.2824    | 11.52     | 175.7     |
| 7         | 0.9843    | 7.278     | 0.5126    | 9.285     | 171.4     |
| 8         | 0.982     | 5.969     | 0.999     | 14.96     | 171.7     |
| 9         | 0.9836    | 10.69     | 0.706     | 12.88     | 167.9     |
| 10        | 0.9821    | 11.46     | 0.2633    | 5.757     | 166.0     |
| 11        | 0.9818    | 11.34     | 0.8473    | 2.871     | 76.63     |
| 12        | 0.966     | 5.02      | 

In [12]:
rf_bo.max

{'target': 0.9846996019601961,
 'params': {'max_depth': 6.178710631830722,
  'max_features': 0.5949274551032339,
  'min_samples_split': 13.30369823056283,
  'n_estimators': 45.03053459891462}}